In [ ]:
import os
import glob
import processing
from qgis.core import QgsApplication

# -------------------------------------------------------
# OPTIONAL: initialize QGIS if running standalone (not inside QGIS)
QgsApplication.setPrefixPath("/usr", True)
qgs = QgsApplication([], False)
qgs.initQgis()
# -------------------------------------------------------

# === Paths ===
dem_dir = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_raw/DEM_tiles/"
out_dir = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/terrain_covariates/"
os.makedirs(out_dir, exist_ok=True)

In [ ]:
# === DEM tile list ===
dem_tiles = sorted(glob.glob(os.path.join(dem_dir, "*.tif")))
print(f"Found {len(dem_tiles)} DEM tiles")

# === Loop over tiles ===
for tile in dem_tiles:
    basename = os.path.splitext(os.path.basename(tile))[0]
    tile_out = os.path.join(out_dir, basename)
    os.makedirs(tile_out, exist_ok=True)

    print(f"\n🔹 Processing tile: {basename}")

In [ ]:
# === DEM tile list ===
dem_tiles = sorted(glob.glob(os.path.join(dem_dir, "*.tif")))
print(f"Found {len(dem_tiles)} DEM tiles")

# === Loop over tiles ===
for tile in dem_tiles:
    basename = os.path.splitext(os.path.basename(tile))[0]
    tile_out = os.path.join(out_dir, basename)
    os.makedirs(tile_out, exist_ok=True)

    print(f"\n🔹 Processing tile: {basename}")

    # --- 1. Slope, Aspect, Curvatures ---
    processing.run("saga:slopeaspectcurvature", {
        'ELEVATION': tile,
        'SLOPE': f'{tile_out}/{basename}_slope.tif',
        'ASPECT': f'{tile_out}/{basename}_aspect.tif',
        'C_GENE': f'{tile_out}/{basename}_curv_general.tif',
        'C_PROF': f'{tile_out}/{basename}_curv_profile.tif',
        'C_PLAN': f'{tile_out}/{basename}_curv_plan.tif',
        'C_TOTA': f'{tile_out}/{basename}_curv_total.tif',
        'C_MAXI': f'{tile_out}/{basename}_curv_max.tif',
        'C_MINI': f'{tile_out}/{basename}_curv_min.tif',
        'C_LONG': f'{tile_out}/{basename}_curv_flowline.tif'
    })

    # --- 2. Hillshade ---
    processing.run("saga:hillshading", {
        'ELEVATION': tile,
        'SHADE': f'{tile_out}/{basename}_hillshade.tif'
    })

    # --- 3. Mid-slope position ---
    processing.run("saga:terrainpositionindex", {
        'DEM': tile,
        'TPI': f'{tile_out}/{basename}_mid_slope_position.tif'
    })

    # --- 4. Normalized height ---
    processing.run("saga:normalizedheights", {
        'DEM': tile,
        'HEIGHT': f'{tile_out}/{basename}_normalized_height.tif'
    })

    # --- 5. Slope height ---
    processing.run("saga:slopeheight", {
        'DEM': tile,
        'SLOPE': f'{tile_out}/{basename}_slope_height.tif'
    })

    # --- 6. Standardized height ---
    processing.run("saga:standardizedheight", {
        'DEM': tile,
        'STDHGT': f'{tile_out}/{basename}_standardized_height.tif'
    })

    # --- 7. Terrain surface convexity ---
    processing.run("saga:terrainsurfaceconvexity", {
        'DEM': tile,
        'CONVEXITY': f'{tile_out}/{basename}_terrain_surface_convexity.tif'
    })

    # --- 8. Terrain surface texture ---
    processing.run("saga:terrainsurfacetexture", {
        'DEM': tile,
        'TEXTURE': f'{tile_out}/{basename}_terrain_surface_texture.tif'
    })

    # --- 9. Valley depth ---
    processing.run("saga:valleydepth", {
        'DEM': tile,
        'VALLEY_DEPTH': f'{tile_out}/{basename}_valley_depth.tif'
    })

    # --- 10. Valley bottom flatness ---
    processing.run("saga:valleybottomflatness", {
        'DEM': tile,
        'FLATNESS': f'{tile_out}/{basename}_valley_flatness.tif'
    })

    print(f"✅ Done with tile: {basename}")

# -------------------------------------------------------
# OPTIONAL: clean up QGIS if running standalone
qgs.exitQgis()
# -------------------------------------------------------


In [ ]:
# ----------------------------------------------
# DEM Terrain Analysis Script for QGIS Python Console
# ----------------------------------------------

# Import QGIS processing library
import processing
from qgis.core import QgsProject

# 1️⃣ Load your DEM
# Replace this path with the path to your DEM file
dem_path = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/covariates_rasters/dem_1km/dem_filledfiltered_1km.tif"
dem_layer = iface.addRasterLayer(dem_path, "DEM")

if not dem_layer.isValid():
    print("❌ DEM failed to load. Check the path!")
else:
    print("✅ DEM loaded successfully")

# 2️⃣ Compute Slope
slope_output = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/qgissaga_test/slope.tif"  # output file
processing.run("gdal:slope", {
    'INPUT': dem_layer,
    'BAND': 1,
    'SCALE': 1.0,
    'OUTPUT': slope_output
})
slope_layer = iface.addRasterLayer(slope_output, "Slope")
print("✅ Slope raster created")

# 3️⃣ Compute Aspect
aspect_output = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/qgissaga_test/aspect.tif"
processing.run("gdal:aspect", {
    'INPUT': dem_layer,
    'BAND': 1,
    'ZERO_FOR_FLAT': True,
    'OUTPUT': aspect_output
})
aspect_layer = iface.addRasterLayer(aspect_output, "Aspect")
print("✅ Aspect raster created")

# 4️⃣ Compute Hillshade
hillshade_output = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/qgissaga_test/hillshade.tif"
processing.run("gdal:hillshade", {
    'INPUT': dem_layer,
    'BAND': 1,
    'Z_FACTOR': 1.0,
    'AZIMUTH': 315,
    'ALTITUDE': 45,
    'COMPUTE_EDGES': True,
    'ZEVENBERG': False,
    'OUTPUT': hillshade_output
})
hillshade_layer = iface.addRasterLayer(hillshade_output, "Hillshade")
print("✅ Hillshade raster created")

# 5️⃣ Done
print("🎉 DEM terrain derivatives created and added to QGIS project!")
